# Baseline Evaluation for BPI Challenge 2012 models

Inside the folder `<project_root>/runs/bpic2012` we have a list of folders named as `<percentage>%`, where `<percentage>` is the percentage of the dataset used for training the model.
In each of these folders we have a folder named as the best model found during the training phase based on the accuracy value.
Inside each of these folders we have the following files:
- `constraints_satisfaction_rate.csv`: a CSV file containing the constraints satisfaction rate for each of the test traces.
- `constraints_satisfactions.csv`: a CSV file containing the constraints satisfaction for each of the test traces.
- `predicted_traces.txt`: a TXT file containing the traces generated by the model for each of the test traces.
- `predictions.csv`: a CSV file containing the predictions step by step for each of the test traces.
- `results.json`: a JSON file containing the results of the evaluation of the model on the test set.

In [1]:
DATASET_NAME = "bpic2012"

In [2]:
from collections import namedtuple
import pathlib

project_root = pathlib.Path("../../..").parent.resolve()

Info = namedtuple("Info", ["model_args", "model_path", "results_path"])

models_path: dict[int, list[Info]] = {}

for dataset_percentage in range(20, 101, 20):
    checkpoints = [
        path
        for path in (project_root / "runs" / DATASET_NAME).rglob(
            f"{dataset_percentage}%/**/*.best_val_acc.pth"
        )
    ]
    results = [
        pathlib.Path(str(checkpoint).removesuffix(".pth")) / "results.json"
        for checkpoint in checkpoints
    ]
    args = [checkpoint.parent / "args.json" for checkpoint in checkpoints]
    models_path[dataset_percentage] = [
        Info(model_args=args, model_path=checkpoint, results_path=result)
        for args, checkpoint, result in zip(args, checkpoints, results)
    ]

## Comparison

In [3]:
import json
import pandas as pd

dataframes = {}

for percentage in models_path:
    dataframes[percentage] = pd.DataFrame(
        columns=[
            "lr",
            "dropout",
            "loss",
            "acc",
            "dld",
            "norm_dld",
            "constraints",
            "constraints_multiplier",
        ]
    )
    for info in models_path[percentage]:
        with open(info.model_args) as f:
            args = json.load(f)
        try:
            with open(info.results_path) as f:
                results = json.load(f)
                dataframes[percentage].loc[info.model_path.parent.name] = [
                    args["learning_rate"],
                    args["model"]["dropout"],
                    results["loss"],
                    results["acc"],
                    results["dld"],
                    results["norm_dld"],
                    args.get("constraints", None),
                    args.get("constraints_multiplier", None),
                ]
        except FileNotFoundError:
            print(f"Missing results for {info.model_path}")

/tmp/ipykernel_2797472/703172264.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[percentage].loc[info.model_path.parent.name] = [
/tmp/ipykernel_2797472/703172264.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframes[percentage].loc[info.model_path.parent.name] = [
/tmp/ipykernel_2797472/703172264.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA colu

In [4]:
for percentage in range(20, 101, 20):
    print("=" * 10 + f" {percentage}% " + "=" * 10)
    display(dataframes[percentage].sort_values("acc", ascending=False))
    print()

========== 20% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250301.1344.no_constraint,0.0001,0.2,1.019818,0.853428,1.332061,0.163858,[],NaN
20250301.1318.no_constraint,0.0001,0.0,1.065801,0.852170,1.343511,0.157479,[],NaN
20250301.1331.no_constraint,0.0010,0.2,1.026502,0.851751,1.347328,0.162482,[],NaN
20250301.1304.no_constraint,0.0010,0.0,1.108723,0.848186,1.379771,0.167899,[],NaN
20250302.1923.constraints,0.0010,0.2,1.125255,0.842525,1.433206,0.171883,[Absence2[A_PREACCEPTED]],1.0
20250302.1824.constraints,0.0001,0.0,1.085774,0.841896,1.437023,0.167283,[Absence2[A_PREACCEPTED]],1.0
20250302.1857.constraints,0.0001,0.0,1.064795,0.840218,1.452290,0.165386,[Absence2[A_PREACCEPTED]],0.1
20250302.1956.constraints,0.0010,0.2,1.054949,0.839379,1.461832,0.170740,[Absence2[A_PREACCEPTED]],0.1



========== 40% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250301.2150.no_constraint,0.0010,0.2,1.002893,0.860049,1.101836,0.137527,[],NaN
20250301.2124.no_constraint,0.0001,0.0,0.985988,0.859625,1.105175,0.136637,[],NaN
20250301.2217.no_constraint,0.0001,0.2,0.967793,0.859519,1.106010,0.139257,[],NaN
20250301.2057.no_constraint,0.0010,0.0,1.015219,0.858989,1.110184,0.140236,[],NaN
20250303.0856.constraints,0.0010,0.2,1.006664,0.857506,1.121870,0.137612,[Absence2[O_SELECTED]],0.1
20250303.0943.constraints,0.0010,0.2,1.004492,0.847971,1.196995,0.143655,[Absence2[O_SELECTED]],1.0



========== 60% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250302.0007.no_constraint,0.0010,0.2,1.026148,0.860172,1.222857,0.138803,[],NaN
20250301.2325.no_constraint,0.0001,0.0,0.995967,0.859585,1.228571,0.137723,[],NaN
20250302.0049.no_constraint,0.0001,0.2,1.011153,0.858867,1.235429,0.138391,[],NaN
20250301.2243.no_constraint,0.0010,0.0,1.061755,0.858541,1.237714,0.141265,[],NaN
20250303.1030.constraints,0.0010,0.2,1.057000,0.850907,1.304571,0.143625,[Absence2[A_PREACCEPTED]],1.0
20250303.1147.constraints,0.0010,0.2,1.057000,0.850907,1.304571,0.143625,[Absence2[A_PREACCEPTED]],1.0



========== 80% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250302.0325.no_constraint,0.0010,0.2,0.977729,0.860966,1.189730,0.136943,[],None
20250302.0422.no_constraint,0.0001,0.2,0.948605,0.859849,1.198869,0.138541,[],None
20250302.0227.no_constraint,0.0001,0.0,0.977471,0.859646,1.201480,0.140530,[],None
20250302.0131.no_constraint,0.0010,0.0,1.009872,0.859138,1.205831,0.139070,[],None



========== 100% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2007.constraints,0.0010,0.2,0.970632,0.864527,1.180150,0.134382,"[Absence2[A_DECLINED], Absence2[A_FINALIZED]]",0.1
20250303.1319.constraints,0.0010,0.2,0.972804,0.864254,1.182529,0.133518,[Absence2[A_DECLINED]],1.0
20250303.1545.constraints,0.0010,0.2,0.969282,0.863086,1.192726,0.135138,[Absence2[A_DECLINED]],0.1
20250301.1813.no_constraint,0.0001,0.2,0.952729,0.863047,1.192386,0.134366,[],NaN
20250301.1550.no_constraint,0.0001,0.0,0.955873,0.862657,1.195785,0.136612,[],NaN
20250301.1702.no_constraint,0.0010,0.2,0.971118,0.862579,1.197145,0.135933,[],NaN
20250301.1434.no_constraint,0.0010,0.0,0.990130,0.860865,1.212101,0.145986,[],NaN
20250303.1754.constraints,0.0010,0.2,0.999016,0.854789,1.265126,0.138297,"[Absence2[A_DECLINED], Absence2[A_FINALIZED]]",1.0


In [5]:
for percentage in range(20, 101, 20):
    print("=" * 10 + f" {percentage}% " + "=" * 10)
    display(dataframes[percentage].sort_values("acc", ascending=False).head(1))
    print()

========== 20% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250301.1344.no_constraint,0.0001,0.2,1.019818,0.853428,1.332061,0.163858,[],NaN



========== 40% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250301.2150.no_constraint,0.001,0.2,1.002893,0.860049,1.101836,0.137527,[],NaN



========== 60% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250302.0007.no_constraint,0.001,0.2,1.026148,0.860172,1.222857,0.138803,[],NaN



========== 80% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250302.0325.no_constraint,0.001,0.2,0.977729,0.860966,1.18973,0.136943,[],None



========== 100% ==========


,lr,dropout,loss,acc,dld,norm_dld,constraints,constraints_multiplier
20250303.2007.constraints,0.001,0.2,0.970632,0.864527,1.18015,0.134382,"[Absence2[A_DECLINED], Absence2[A_FINALIZED]]",0.1
